# Classifier Comparison

In [1]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np



In [2]:
df = pd.read_csv('Downloads/features_30_sec.csv', sep=',')
df = df.drop(['filename'], axis = 1)

In [3]:

df


,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,3805.839606,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,3550.522098,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,3042.260232,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,2184.745799,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,3579.757627,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,661794,0.352063,0.080487,0.079486,0.000345,2008.149458,282174.689224,2106.541053,88609.749506,4253.557033,...,45.050526,-13.289984,41.754955,2.484145,36.778877,-6.713265,54.866825,-1.193787,49.950665,rock
996,661794,0.398687,0.075086,0.076458,0.000588,2006.843354,182114.709510,2068.942009,82426.016726,4149.338328,...,33.851742,-10.848309,39.395096,1.881229,32.010040,-7.461491,39.196327,-2.795338,31.773624,rock
997,661794,0.432142,0.075268,0.081651,0.000322,2077.526598,231657.968040,1927.293153,74717.124394,4031.405321,...,33.597008,-12.845291,36.367264,3.440978,36.001110,-12.588070,42.502201,-2.106337,29.865515,rock
998,661794,0.362485,0.091506,0.083860,0.001211,1398.699344,240318.731073,1818.450280,109090.207161,3015.631004,...,46.324894,-4.416050,43.583942,1.556207,34.331261,-5.041897,47.227180,-3.590644,41.299088,rock


In [4]:
y_p = df.loc[:,'label']
X_p = df.drop(['label'], axis=1)

stdscale = StandardScaler(with_mean=False, with_std=False)
X_p = stdscale.fit_transform(X_p)


# Linear Method

## Logistic Regression

In [5]:
%%time

lr = LogisticRegression(max_iter = 10000)

# Set up possible values of parameters to optimize over
# In this case, 12 possible values of C
param_grid = {
    "C": [10**(-6), 10**(-5), 10**(-4), 10**(-3), 10**(-2), 10**(-1),
        0, 10**(0), 10**(1), 10**(2), 10**(3), 10**(4)]}


# Array to store scores
lrscores = np.zeros(3)
best_lrmodel = []

# Loop for each trial
for i in range(3):

     # Create test/train
    X_train, X_test, y_train, y_test = train_test_split(X_p, y_p,
                                                    train_size=0.8,
                                                    random_state=3,
                                                    stratify=y_p)
    # Create inner cv
    inner_cv = KFold(n_splits=5, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    clf = GridSearchCV(lr, param_grid, cv=inner_cv, n_jobs = -1)
    clf.fit(X_train, y_train)
    lrscores[i] = clf.best_score_
    best_lrmodel.append(clf.best_estimator_.fit(X_train, y_train))
    

/Users/elliott/.pyenv/versions/3.8.6/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/elliott/.pyenv/versions/3.8.6/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

CPU times: user 3min 2s, sys: 27.1 s, total: 3min 29s
Wall time: 3min 54s


/Users/elliott/.pyenv/versions/3.8.6/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [6]:
lrscores


array([0.46375, 0.4775 , 0.49   ])

In [7]:
best_lrmodel


[LogisticRegression(C=1000, max_iter=10000),
 LogisticRegression(C=0.0001, max_iter=10000),
 LogisticRegression(C=1000, max_iter=10000)]

In [8]:
clf.cv_results_


{'mean_fit_time': array([1.60153904e+01, 1.62372821e+01, 1.48585456e+01, 1.41863004e+01,
        1.48799084e+01, 1.43020267e+01, 6.41303062e-03, 1.44630464e+01,
        1.42250195e+01, 1.37319678e+01, 1.17265543e+01, 8.29174910e+00]),
 'std_fit_time': array([1.60224297e-01, 7.66420975e-01, 1.06834898e+00, 1.89545598e-01,
        7.64862303e-01, 1.66807190e-01, 1.09606433e-04, 3.78360140e-01,
        9.82506717e-01, 1.80831499e-01, 2.42761670e+00, 1.29788366e-01]),
 'mean_score_time': array([0.00105906, 0.00101547, 0.00123577, 0.00100183, 0.00099168,
        0.0009831 , 0.        , 0.00127721, 0.0009831 , 0.00102482,
        0.00061622, 0.00059443]),
 'std_score_time': array([8.84804722e-05, 3.85408334e-05, 4.30177035e-04, 3.06079688e-05,
        3.62203572e-05, 2.39449181e-05, 0.00000000e+00, 6.00646786e-04,
        1.25668079e-05, 1.31756811e-04, 1.72463807e-04, 1.78570162e-04]),
 'param_C': masked_array(data=[1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1, 0, 1, 10, 100,
                    

## Logistic Regression: Train and Test with Optimal Params

In [9]:
#Train:

X_train, X_test, y_train, y_test = train_test_split(X_p, y_p,
                                                    train_size=0.80,
                                                    random_state=1,
                                                    stratify=y_p)

stdscale.fit(X_train)

X_train = stdscale.transform(X_train)
X_test = stdscale.transform(X_test)

#optimized parameters
classifier = LogisticRegression(C = 1000, max_iter = 10000)
classifier.fit(X_train, y_train)


/Users/elliott/.pyenv/versions/3.8.6/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=1000, max_iter=10000)

In [10]:
#Test:

pred = classifier.predict(X_test)
matches = (pred==y_test).sum()

accuracy = matches / pred.shape
accuracy


array([0.465])

# Non-Linear Method

## kNN: Parameter Search

In [15]:
%%time

my_list = list(range(1, 501, 50))

# Set up possible values of parameters to optimize over
# In this case, 10 values of K, 2 values for weights,
# three potential algorithms and 3 leaf sizes.
param_grid = {
    "n_neighbors": my_list,
    'weights': ['uniform', 'distance'],
    "algorithm": ['auto', 'ball_tree', 'kd_tree'],
    "leaf_size": [20, 30, 40]}

# Array to store scores
knnscores = np.zeros(3)
best_knnmodel = []

# Loop for each trial
for i in range(3): #NUM_TRIALS

     # Create test/train
    X_train, X_test, y_train, y_test = train_test_split(X_p, y_p,
                                                    train_size=0.8,
                                                    random_state=1,
                                                    stratify=y_p)
    # Create inner cv
    inner_cv = KFold(n_splits=5, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    clf = GridSearchCV(KNeighborsClassifier(), param_grid, cv=inner_cv, n_jobs = -1)
    clf.fit(X_train, y_train)
    knnscores[i] = clf.best_score_
    best_knnmodel.append(clf.best_estimator_.fit(X_train, y_train))
    

CPU times: user 1.89 s, sys: 160 ms, total: 2.05 s
Wall time: 5.88 s


In [16]:
knnscores


array([0.28   , 0.30125, 0.3075 ])

In [17]:
best_knnmodel


[KNeighborsClassifier(leaf_size=20, n_neighbors=151, weights='distance'),
 KNeighborsClassifier(leaf_size=20, n_neighbors=351, weights='distance'),
 KNeighborsClassifier(leaf_size=20, n_neighbors=51, weights='distance')]

In [18]:
clf.cv_results_


{'mean_fit_time': array([0.00202713, 0.00189376, 0.0021625 , 0.00225482, 0.0021801 ,
        0.00254898, 0.00266528, 0.00281243, 0.00242214, 0.00236964,
        0.00236969, 0.00230818, 0.00258579, 0.00200434, 0.00128107,
        0.00172663, 0.00172658, 0.00176268, 0.00160508, 0.0017715 ,
        0.0028729 , 0.00321693, 0.00298734, 0.00291281, 0.00284133,
        0.00303226, 0.00263834, 0.00266819, 0.00279303, 0.00258923,
        0.00258083, 0.00260386, 0.00290847, 0.0031559 , 0.00182905,
        0.00179844, 0.00184455, 0.00182862, 0.00209632, 0.00202565,
        0.00282288, 0.00284381, 0.00359607, 0.00291271, 0.00289159,
        0.00290523, 0.00290322, 0.00288081, 0.00280437, 0.00286174,
        0.00294876, 0.00282235, 0.00274673, 0.0033669 , 0.00171022,
        0.00179243, 0.00185785, 0.00158057, 0.00162334, 0.00172091,
        0.0027669 , 0.00330176, 0.00372629, 0.00283437, 0.00287838,
        0.00290642, 0.00289669, 0.00278034, 0.00344639, 0.00282035,
        0.00277276, 0.00298743,

## kNN: Train and Test with Optimal Params

In [19]:
#Train:

X_train, X_test, y_train, y_test = train_test_split(X_p, y_p,
                                                    train_size=0.80,
                                                    random_state=1,
                                                    stratify=y_p)

stdscale.fit(X_train)

X_train = stdscale.transform(X_train)
X_test = stdscale.transform(X_test)

#optimized parameters
classifier = KNeighborsClassifier(n_neighbors=51,
                                  weights='distance')
classifier.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=51, weights='distance')

In [20]:
#Test:

pred = classifier.predict(X_test)
matches = (pred==y_test).sum()

accuracy = matches / pred.shape
accuracy


array([0.28])

# Ensemble Method

## Random Forest: Parameter Search

In [21]:
%%time

rf = RandomForestClassifier(n_estimators=1024)

# Set up possible values of parameters to optimize over
# In this case, 8 max_feature values
param_grid = {
    "max_features": [1, 2, 4, 6, 8, 12, 16, 20]}

# Array to store scores
rfscores = np.zeros(3)
best_rfmodel = []

# Loop for each trial
for i in range(3):

     # Create test/train
    X_train, X_test, y_train, y_test = train_test_split(X_p, y_p,
                                                    train_size=0.8,
                                                    random_state=2,
                                                    stratify=y_p)
    # Create inner cv
    inner_cv = KFold(n_splits=5, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    clf = GridSearchCV(RandomForestClassifier(), param_grid, cv=inner_cv, n_jobs = -1)
    clf.fit(X_train, y_train)
    rfscores[i] = clf.best_score_
    best_rfmodel.append(clf.best_estimator_.fit(X_train, y_train))
    


CPU times: user 3.28 s, sys: 34.4 ms, total: 3.31 s
Wall time: 9.6 s


In [22]:
rfscores


array([0.79   , 0.78375, 0.775  ])

In [23]:
best_rfmodel


[RandomForestClassifier(max_features=16),
 RandomForestClassifier(max_features=12),
 RandomForestClassifier(max_features=12)]

In [24]:
clf.cv_results_


{'mean_fit_time': array([0.23200054, 0.2948998 , 0.38339229, 0.43104992, 0.47556448,
        0.59285536, 0.70158343, 0.67200928]),
 'std_fit_time': array([0.00682509, 0.01009461, 0.00689627, 0.01100309, 0.00195177,
        0.00719719, 0.01145282, 0.05385502]),
 'mean_score_time': array([0.01753812, 0.02128177, 0.01623001, 0.01585641, 0.01595674,
        0.01526537, 0.01319051, 0.0094254 ]),
 'std_score_time': array([0.00178513, 0.00033257, 0.00067606, 0.00130834, 0.000771  ,
        0.00058734, 0.00197986, 0.00035958]),
 'param_max_features': masked_array(data=[1, 2, 4, 6, 8, 12, 16, 20],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_features': 1},
  {'max_features': 2},
  {'max_features': 4},
  {'max_features': 6},
  {'max_features': 8},
  {'max_features': 12},
  {'max_features': 16},
  {'max_features': 20}],
 'split0_test_score': array([0.71875, 0.7125 , 0.7375 , 0.76875, 0.7875 , 0.8

## Random Forest: Train and Test with Optimal Params

In [25]:
#Train:

X_train, X_test, y_train, y_test = train_test_split(X_p, y_p,
                                                    train_size=0.80,
                                                    random_state=1,
                                                    stratify=y_p)

stdscale.fit(X_train)

X_train = stdscale.transform(X_train)
X_test = stdscale.transform(X_test)

#optimized parameters
classifier = RandomForestClassifier(max_features=12)
classifier.fit(X_train, y_train)


RandomForestClassifier(max_features=12)

In [26]:
#Test:

pred = classifier.predict(X_test)
matches = (pred==y_test).sum()

accuracy = matches / pred.shape
accuracy


array([0.73])

## Neural Network: Parameter Search

In [27]:
%%time

NN = MLPClassifier()

# Set up possible values of parameters to optimize over
# In this case, there are many
param_grid = {
    "solver": ['lbfgs', 'sgd', 'adam'],
    "hidden_layer_sizes": [50, 100, 150],
    "activation": ['logistic','relu'],
    "learning_rate": ['constant', 'invscaling', 'adaptive']}


# Array to store scores
nnscores = np.zeros(3)
best_nnmodel = []

# Loop for each trial
for i in range(3):

     # Create test/train
    X_train, X_test, y_train, y_test = train_test_split(X_p, y_p,
                                                    train_size=0.8,
                                                    random_state=3,
                                                    stratify=y_p)
    # Create inner cv
    inner_cv = KFold(n_splits=5, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    clf = GridSearchCV(NN, param_grid, cv=inner_cv, n_jobs = -1)
    clf.fit(X_train, y_train)
    nnscores[i] = clf.best_score_
    best_nnmodel.append(clf.best_estimator_.fit(X_train, y_train))
    

/Users/elliott/.pyenv/versions/3.8.6/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/elliott/.pyenv/versions/3.8.6/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/elliott/.pyenv/versions/3.8.6/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWa

CPU times: user 1min 6s, sys: 18 s, total: 1min 24s
Wall time: 1min 27s


/Users/elliott/.pyenv/versions/3.8.6/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [28]:
nnscores


array([0.29375, 0.28375, 0.2975 ])

In [29]:
best_nnmodel


[MLPClassifier(activation='logistic', hidden_layer_sizes=100,
               learning_rate='invscaling', solver='lbfgs'),
 MLPClassifier(activation='logistic', hidden_layer_sizes=150,
               learning_rate='invscaling', solver='lbfgs'),
 MLPClassifier(activation='logistic', hidden_layer_sizes=150, solver='lbfgs')]

In [30]:
clf.cv_results_


{'mean_fit_time': array([1.11279516, 1.27592349, 1.35177059, 0.92226501, 0.57143426,
        1.13943448, 1.26693268, 1.31065617, 1.48941102, 2.88939056,
        1.82277884, 1.87338452, 2.95988569, 0.62192869, 1.26921744,
        2.33940139, 1.79030633, 1.96940823, 4.27898664, 2.49687099,
        2.6843132 , 4.64824171, 1.33528886, 2.26306973, 4.80885544,
        2.39703364, 2.17821779, 0.08800201, 0.08382673, 0.37501698,
        0.08845439, 0.08289933, 0.30606041, 0.08739886, 0.42535553,
        0.27597775, 0.12845111, 0.10507298, 0.46874743, 0.12198243,
        0.09974465, 0.54157972, 0.12303338, 0.56737123, 0.29199023,
        0.16694341, 0.13533149, 0.69034767, 0.18698325, 0.13152695,
        0.53170547, 0.17430978, 0.57672839, 0.46955695]),
 'std_fit_time': array([0.45040784, 0.00777996, 0.09810455, 0.21264975, 0.40743131,
        0.4501891 , 0.2276283 , 0.04369236, 0.18927961, 0.55059499,
        0.09540394, 0.23751915, 0.27126731, 0.17089548, 0.64409602,
        0.48409084, 0.012

## Neural Network: Train and Test with Optimal Params

In [31]:
#Train:

X_train, X_test, y_train, y_test = train_test_split(X_p, y_p,
                                                    train_size=0.80,
                                                    random_state=1,
                                                    stratify=y_p)

stdscale.fit(X_train)

X_train = stdscale.transform(X_train)
X_test = stdscale.transform(X_test)

#optimized parameters
classifier = MLPClassifier(activation='logistic',
                           hidden_layer_sizes=100,
                           learning_rate='invscaling',
                           solver='lbfgs')
classifier.fit(X_train, y_train)


/Users/elliott/.pyenv/versions/3.8.6/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(activation='logistic', hidden_layer_sizes=100,
              learning_rate='invscaling', solver='lbfgs')

In [32]:
#Test:

pred = classifier.predict(X_test)
matches = (pred==y_test).sum()

accuracy = matches / pred.shape
accuracy


array([0.27])